<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023/blob/main/Python/LINEBot/LINEBot_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python×LINEBot（基本編）

In [ ]:
# 最初に実行してください
!pip install line-bot-sdk
!pip install pyngrok
!pip install python-dotenv

## 初めに

- [LINE Developersコンソール](https://developers.line.biz/console/)にアクセスし、チャネルを作成してください。
手順については[LINE Developersコンソールでチャネルを作成する](https://developers.line.me/ja/docs/messaging-api/getting-started/)を参考にしてください。


  - チャネルの「チャネル基本設定」で、
チャネルアクセストークンとチャネルシークレットを取得してください。


  - 取得したチャネルアクセストークンとチャネルシークレットを、
"Your Channel Access Token"と"Your Channel Secret"の部分に置き換えてください。

  - WebHookURL を、 `Public URL: https://xx-xx-xx-xx-xx..ngrok.io`となっているところをふまえて
`https://xx-xx-xx-xx-xx..ngrok.io/callback` に変更してください。


In [6]:
# 自分のトークンを設定してください
import os

directory = 'content/info'
if not os.path.exists(directory):
    os.makedirs(directory)

file_path = os.path.join(directory, '.env')
with open(file_path, 'w') as f:
    f.write('CHANNEL_ACCESS_TOKEN = Your Channel Access Token\n')
    f.write('CHANNEL_SECRET = Your Channel Secret\n')

## 例1　テキストのオウム返し

何かメッセージを送った時に、Botがオウム返しをするようにします。

テキストメッセージについては[公式のドキュメント：テキストメッセージ](https://developers.line.biz/ja/docs/messaging-api/message-types/#text-messages)などを参考にしてください。

＜実行イメージ＞

![](https://imgur.com/o0x0z6D.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok

app = Flask(__name__)
line_bot_api = LineBotApi('Your Channel Access Token')  # LineBotApiオブジェクトを作成し、Channel Access Tokenを指定
handler = WebhookHandler('Your Channel Secret')  # WebhookHandlerオブジェクトを作成し、Channel Secretを指定

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    reply_text = event.message.text  # 受信したテキストメッセージを取得
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)  # 受信したテキストメッセージをそのまま返信
    )
    print("返信完了!!\ntext:", event.message.text)  # 返信が完了したことを表示

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行


## 例2 画像データの取り扱い

Botは画像を返答することができます。

ここでは、ユーザーが送った画像を（Colabで）保存し、
画像が送信されたことが分かる画像を返信します。

画像メッセージについては[公式のドキュメント：画像メッセージ](https://developers.line.biz/ja/docs/messaging-api/message-types/#image-messages)などを参考にしてください。

＜実行イメージ＞

![](https://imgur.com/gPrWbnk.png)

In [ ]:
import os
from pathlib import Path
from datetime import datetime
import pytz

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    reply_text = event.message.text  # 受信したテキストメッセージを取得
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)  # 受信したテキストメッセージをそのまま返信
    )
    print("返信完了!!\ntext:", event.message.text)  # 返信が完了したことを表示

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id  # 受信した画像メッセージのIDを取得
    message_content = line_bot_api.get_message_content(message_id)  # 画像メッセージの内容を取得

    # 画像を保存する一時ファイルのパスを作成
    tmp_dir = "image"
    os.makedirs(tmp_dir, exist_ok=True)
    # タイムゾーンを設定
    tz = pytz.timezone('Asia/Tokyo')
    # 東京の現在の日時情報を取得
    now = datetime.now(tz)
    # 日時情報からファイル名を生成
    tmp_path = f"{tmp_dir}/{now.strftime('%Y%m%d%H%M%S')}.jpg"

    # 画像を一時ファイルとして保存
    with open(tmp_path, "wb") as f:
        for chunk in message_content.iter_content():
            f.write(chunk)

    # テキトーな画像を返す（imgurにアップしたもの）
    # 画像のオウム返しは難しいようです
    line_bot_api.reply_message(
        event.reply_token,
        ImageSendMessage(original_content_url=f"https://imgur.com/mqd9pWE.png", preview_image_url=f"https://imgur.com/mqd9pWE.png")
    )
    print("返信完了!!\nimage:", tmp_path)  # 返信が完了したことと一時ファイルのパスを表示


ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

＜画像の保存の確認＞

![](https://imgur.com/fK9djR0.png)

## 例3 送信回数のカウント

Botとのやりとりを継続するには、状態を管理することが必要になります。

Pythonのglobal変数をうまく利用することで、管理できるようになります。

ここでは、Botとのやりとりの回数をglobal変数`count`で管理します。

＜実行イメージ＞

![](https://imgur.com/yAzolgb.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

# カウント用のグローバル変数
count = 0

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global count  # グローバル変数を参照するために宣言
    count += 1  # カウントをインクリメント

    reply_text = str(count)  # カウントを文字列に変換して返信
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)
    )
    print("返信完了!!\ntext:", event.message.text)

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

## 例4 ひとつ前のメッセージを返信

状態管理の続きです。

ここでは、Botに送信した内容をglobal変数`count`で管理します。

＜実行イメージ＞

![](https://imgur.com/35bEO28.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

# カウント用のグローバル変数
count = "スタート"

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global count  # グローバル変数を参照するために宣言

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=count)  # グローバル変数の値を返信
    )
    count = event.message.text  # 受信したメッセージでグローバル変数を更新
    print("返信完了!!\ntext:", event.message.text)

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行


## 例5 電卓プログラム

簡単な四則演算をLINEBotで行えるようにします。

＜実行イメージ＞

![](https://imgur.com/SlnkQRQ.png)　![](https://imgur.com/tMptQuu.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

# カウント用のグローバル変数
session = {}  # 電卓セッションの情報を保持する辞書
not_dentaku = True  # 電卓モードかどうかを示すフラグ

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

# 計算結果を返す関数
def calculate_result():
    num1 = session['num1']
    operator = session['operator']
    num2 = session['num2']

    if operator == '+':
        result = num1 + num2
    elif operator == '-':
        result = num1 - num2
    elif operator == '*':
        result = num1 * num2
    elif operator == '/':
        result = num1 / num2
    else:
        result = None

    result = round(result, 2)
    return f"{num1} {operator} {num2} = {str(result)}"

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global session  # 入力情報を管理するグローバル変数
    global not_dentaku  # 電卓モードかどうかを管理するグローバル変数

    # テキストメッセージを返す関数
    def output_text(output_message):
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=output_message)
        )

    # ユーザーが入力したメッセージを変数に代入
    input_message = event.message.text
    # 無限ループ
    while True:
        # "電卓"と入力された場合
        if input_message == "電卓":
            # 電卓モードではない場合
            if not_dentaku:
                # 電卓モードのスタート
                reply_message = "電卓を開始します\n最初の数値を入力してください"
                output_text(reply_message)
                not_dentaku = False
                break
            # すでに電卓モードである場合
            else:
                # 電卓モードの終了
                reply_message = "電卓を終了します"
                output_text(reply_message)
                not_dentaku = True
                break

        # "電卓"以外が入力された場合
        else:
            # 電卓モードではない場合
            if not_dentaku:
                # オウム返し
                reply_message = input_message
                output_text(reply_message)
                break

        # まだ一つ目の数が入力されていない場合
        if 'num1' not in session:
            # 数値かどうかを判断
            try:
                # 数値の場合は処理続行
                session['num1'] = float(input_message)
                reply_message = '演算子を入力してください\n (+, -, *, /)'
                output_text(reply_message)
                break
            except ValueError:
                # 数値ではない場合は再度入力させる
                reply_message = '無効な数値です\n最初の数値を入力してください'
                output_text(reply_message)
                break

        # まだ演算子が入力されていない場合
        if 'operator' not in session:
            # 演算子かどうかを判断
            if input_message in ['+', '-', '*', '/']:
                # 演算子の場合は処理続行
                session['operator'] = input_message
                reply_message = '2番目の数値を入力してください'
                output_text(reply_message)
                break
            else:
                # 演算子ではない場合は再度入力させる
                reply_message = '無効な演算子です\n演算子を入力してください (+, -, *, /)'
                output_text(reply_message)
                break

        # まだ二つ目の数が入力されていない場合
        if 'num2' not in session:
            # ゼロ除算かどうかを判断
            if session['operator'] == "/" and input_message == "0":
                reply_message = '無効な数値です\n2番目の数値を入力してください'
                output_text(reply_message)
                break

            # 数値かどうかを判断
            try:
                # 数値の場合は処理続行
                session['num2'] = float(input_message)

                # これまでの入力データを使って計算
                reply_message = calculate_result()

                # 追加のメッセージを定義
                plus_message = "続けて最初の数字を入力してください\n電卓を終了したい場合は「電卓」と入力してください"

                # 2つのメッセージを返信
                line_bot_api.reply_message(
                    event.reply_token,
                    [TextSendMessage(text=reply_message), TextSendMessage(text=plus_message)]
                )

                # セッション情報をクリア
                session = {}
                break

            except ValueError:
                # 数値ではない場合は再度入力させる
                reply_message = '無効な数値です\n2番目の数値を入力してください'
                output_text(reply_message)
                break

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

## 最後に

基本編の資料はこれで終了です。
プロメン最終成果物の参考にしてください。

もっと機能を追加したい方は応用編の資料を参照してください。